In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,LSTM,Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Read Dataset

In [2]:
data = pd.read_csv('Sentiment.csv') 
# Keeping only the neccessary columns 
data = data[['text','sentiment']] 

# Process Data

In [3]:
data['text'] = data['text'].apply(lambda x: x.lower()) 
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) 
for idx, row in data.iterrows(): 
    row[0] = row[0].replace('rt',' ') 
max_features = 2000 
tokenizer = Tokenizer(num_words=max_features, split=' ') 
tokenizer.fit_on_texts(data['text'].values) 
X = tokenizer.texts_to_sequences(data['text'].values) 
X = pad_sequences(X)  

# Create LSTM Model

In [4]:
embed_dim = 128
lstm_out = 196 
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1])) 
#model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) 

#These parameters allow for GPU usage for LSTM model, speeding up the training dramatically, otherwise GPU stopped working for this model.
model.add(LSTM(lstm_out, activation='tanh',recurrent_activation='sigmoid',recurrent_dropout=0,unroll=False,use_bias=True)) 

# 3 outputs
model.add(Dense(3,activation='softmax')) 
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) 
(model.summary()) 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________


# Categorize Output

In [5]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment']) 
Y = to_categorical(integer_encoded) 

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42) 
print(X_train.shape)
print(Y_train.shape) 
print(X_test.shape)
print(Y_test.shape) 

(9293, 28)
(9293, 3)
(4578, 28)
(4578, 3)


In [7]:
with tf.device('/GPU:0'):
    model.fit(X_train, Y_train, epochs = 4) 

Epoch 1/4
291/291 [==============================] - 8s 5ms/step - loss: 0.8208 - accuracy: 0.6477
Epoch 2/4
291/291 [==============================] - 2s 6ms/step - loss: 0.6711 - accuracy: 0.7146
Epoch 3/4
291/291 [==============================] - 2s 6ms/step - loss: 0.6103 - accuracy: 0.7453
Epoch 4/4
291/291 [==============================] - 1s 5ms/step - loss: 0.5540 - accuracy: 0.7712


In [8]:
score,acc = model.evaluate(X_test, Y_test) 
print("score: %.2f" % (score)) 
print("acc: %.2f" % (acc)) 

144/144 [==============================] - 1s 3ms/step - loss: 0.8030 - accuracy: 0.6651
score: 0.80
acc: 0.67


# Creating prediction string and processing the data with previous tokenizer

In [9]:
predicion_input = ['A lot of good things are happening. We are respected again throughout the world, and that\'s a great thing.@realDonaldTrump']

In [10]:
predicion_input = tokenizer.texts_to_sequences(predicion_input)

#Setting the shape to match the input
predicion_input = pad_sequences(predicion_input,maxlen=28)

In [11]:
model.predict(predicion_input)

array([[0.15068953, 0.07452668, 0.7747838 ]], dtype=float32)

In [12]:
#Using argmax and class labels to print out the prediction 
labelencoder.classes_[np.argmax(model.predict(predicion_input))]

'Positive'

# Read Spam Data

In [13]:
spam_data = pd.read_csv('spam.csv',encoding='latin-1') 
# Keeping only the neccessary columns 
spam_data = spam_data[['v1','v2']] 

# Process data

In [14]:
spam_data['v2'] = spam_data['v2'].apply(lambda x: x.lower()) 
spam_data['v2'] = spam_data['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) 
for idx, row in spam_data.iterrows(): 
    row[0] = row[0].replace('rt',' ') 
max_features = 2000 
tokenizer_spam = Tokenizer(num_words=max_features, split=' ') 
tokenizer_spam.fit_on_texts(spam_data['v2'].values) 
X = tokenizer_spam.texts_to_sequences(spam_data['v2'].values) 
X = pad_sequences(X)  

# Create LSTM model with new embed dim

In [15]:
embed_dim = 152
lstm_out = 196 
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1])) 
#model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) 
#Faster for GPU
model.add(LSTM(lstm_out, activation='tanh',recurrent_activation='sigmoid',recurrent_dropout=0,unroll=False,use_bias=True)) 
# 2 outputs this time
model.add(Dense(2,activation='softmax')) 
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) 
(model.summary()) 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 152, 152)          304000    
                                                                 
 lstm_1 (LSTM)               (None, 196)               273616    
                                                                 
 dense_1 (Dense)             (None, 2)                 394       
                                                                 
Total params: 578,010
Trainable params: 578,010
Non-trainable params: 0
_________________________________________________________________


# Categorize data

In [16]:
labelencoder = LabelEncoder()
integer_encoded_spam = labelencoder.fit_transform(spam_data['v1']) 
Y = to_categorical(integer_encoded_spam) 

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42) 
print(X_train.shape)
print(Y_train.shape) 
print(X_test.shape)
print(Y_test.shape) 

(3733, 152)
(3733, 2)
(1839, 152)
(1839, 2)


In [18]:
with tf.device('/GPU:0'):
    model.fit(X_train, Y_train, epochs = 10) 

Epoch 1/10
117/117 [==============================] - 2s 11ms/step - loss: 0.1610 - accuracy: 0.9475
Epoch 2/10
117/117 [==============================] - 1s 11ms/step - loss: 0.0350 - accuracy: 0.9890
Epoch 3/10
117/117 [==============================] - 1s 12ms/step - loss: 0.0145 - accuracy: 0.9960
Epoch 4/10
117/117 [==============================] - 1s 11ms/step - loss: 0.0059 - accuracy: 0.9981
Epoch 5/10
117/117 [==============================] - 1s 12ms/step - loss: 0.0649 - accuracy: 0.9882
Epoch 6/10
117/117 [==============================] - 1s 11ms/step - loss: 0.0091 - accuracy: 0.9968
Epoch 7/10
117/117 [==============================] - 1s 11ms/step - loss: 0.0032 - accuracy: 0.9989
Epoch 8/10
117/117 [==============================] - 1s 11ms/step - loss: 0.0017 - accuracy: 0.9995
Epoch 9/10
117/117 [==============================] - 1s 11ms/step - loss: 0.0012 - accuracy: 0.9995
Epoch 10/10
117/117 [==============================] - 1s 12ms/step - loss: 6.7289e-04 - ac

In [19]:
score,acc = model.evaluate(X_test, Y_test) 
print("score: %.2f" % (score)) 
print("acc: %.2f" % (acc)) 

58/58 [==============================] - 1s 5ms/step - loss: 0.1086 - accuracy: 0.9810
score: 0.11
acc: 0.98


# Transfer Learning

##### Running example from https://colab.research.google.com/github/keras-team/keras-io/blob/master/guides/ipynb/transfer_learning.ipynb#scrollTo=rUEK2sB5_tkt

In [20]:
# Import a cat/dog dataset with 25000 images
import tensorflow_datasets as tfds


tfds.disable_progress_bar()

#Use a smaller amount of the dataset
train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,  # Include labels
)

# Scale the images

In [21]:
size = (150, 150)

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

In [22]:
batch_size = 32

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

# Rotate images so they are slightly different

In [23]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

# Create model with imagenet weights

In [24]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 150, 150, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 150, 150, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

In [25]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

with tf.device('/GPU:0'):
    model.fit(train_ds, epochs=4, validation_data=validation_ds)

Epoch 1/4
291/291 [==============================] - 42s 129ms/step - loss: 0.1713 - binary_accuracy: 0.9257 - val_loss: 0.0855 - val_binary_accuracy: 0.9690
Epoch 2/4
291/291 [==============================] - 79s 271ms/step - loss: 0.1092 - binary_accuracy: 0.9541 - val_loss: 0.0760 - val_binary_accuracy: 0.9725
Epoch 3/4
291/291 [==============================] - 20s 68ms/step - loss: 0.1146 - binary_accuracy: 0.9535 - val_loss: 0.0745 - val_binary_accuracy: 0.9729
Epoch 4/4
291/291 [==============================] - 19s 66ms/step - loss: 0.0993 - binary_accuracy: 0.9596 - val_loss: 0.0753 - val_binary_accuracy: 0.9733
